引入工具库

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

计算信息熵函数

In [2]:
def cal_entropy(text):
    h = 0.0
    sum = 0
    letter = [0] * 26
    text = text.lower()
    for i in range(len(text)):
        if text[i].isalpha():
            letter[ord(text[i]) - ord('a')] += 1
            sum += 1
    for i in range(26):
        p = 1.0 * letter[i] / sum
        if p > 0:
            h += -(p * math.log(p, 2))
    return h

加载数据集

In [3]:
df = pd.read_csv('dns_train&test.csv')

分别计算域名信息熵

In [5]:
temp = []
for each in df['dns.qry.name']:
    temp.append(cal_entropy(each))
df['dns.qry.name.entropy'] = temp
order = ['frame.time_relative', 'ip.src', 'ip.dst', 'dns.qry.name', 'ip.proto',
       'frame.len', 'dns.qry.name.len', 'dns.qry.type', 'dns.qry.name.entropy', 'Type']
df = df[order]
print(df)

     frame.time_relative           ip.src           ip.dst  \
0               0.000000  192.168.113.129    192.168.113.2   
1               1.014907  192.168.113.129    192.168.113.2   
2               2.029145  192.168.113.129    192.168.113.2   
3               2.057082    192.168.113.2  192.168.113.129   
4               2.057309    192.168.113.2  192.168.113.129   
..                   ...              ...              ...   
193            28.968437    192.168.113.2  192.168.113.128   
194            29.486746  192.168.113.128    192.168.113.2   
195            29.486867  192.168.113.128    192.168.113.2   
196            29.491123    192.168.113.2  192.168.113.128   
197            29.491256    192.168.113.2  192.168.113.128   

                   dns.qry.name  ip.proto  frame.len  dns.qry.name.len  \
0    477124f2.ns2.heart1ess.xyz        17         86                26   
1    477124f2.ns2.heart1ess.xyz        17         86                26   
2    477124f2.ns2.heart1ess.xyz  

切分数据集

In [6]:
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
df.head()

,frame.time_relative,ip.src,ip.dst,dns.qry.name,ip.proto,frame.len,dns.qry.name.len,dns.qry.type,dns.qry.name.entropy,Type,is_train
0,0.000000,192.168.113.129,192.168.113.2,477124f2.ns2.heart1ess.xyz,17,86,26,1,3.324863,DNS-Tunnel,True
1,1.014907,192.168.113.129,192.168.113.2,477124f2.ns2.heart1ess.xyz,17,86,26,1,3.324863,DNS-Tunnel,True
2,2.029145,192.168.113.129,192.168.113.2,477124f2.ns2.heart1ess.xyz,17,86,26,1,3.324863,DNS-Tunnel,False
3,2.057082,192.168.113.2,192.168.113.129,477124f2.ns2.heart1ess.xyz,17,102,26,1,3.324863,DNS-Tunnel,True
4,2.057309,192.168.113.2,192.168.113.129,477124f2.ns2.heart1ess.xyz,17,102,26,1,3.324863,DNS-Tunnel,True


In [7]:
train, test = df[df['is_train']==True], df[df['is_train']==False]

选择模型

In [9]:
features = df.columns[4:9]
clf = RandomForestClassifier(n_jobs=2)
y, _ = pd.factorize(train['Type'])
print(y)
print(_)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Index(['DNS-Tunnel', 'Normal-DNS'], dtype='object')


训练模型

In [10]:
clf.fit(train[features], y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

模型预测

In [11]:
preds = _[clf.predict(test[features])]
pd.crosstab(test['Type'], preds, rownames=['actual'], colnames=['preds'])

preds,DNS-Tunnel,Normal-DNS
actual,,
DNS-Tunnel,48,0
Normal-DNS,0,11
